In [3]:
import pandas as pd 
import numpy as np
import seaborn as sns
import re
import os
from matplotlib import pyplot as plt
#from emfdscore.scoring import score_docs
import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter
from torchtext.vocab import Vocab
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [4]:
df= pd.read_csv('transcripts/emfd_out.csv')
df = df[df['count']>=60]
y = df.pop('Y')
df.insert(0, 'Y', y)
data = df.drop(df.loc[:, 'care_sent':'count'].columns, axis = 1)
data.reset_index(inplace=True)
data.drop(['index'], axis = 1, inplace=True)

In [5]:
train, valid, test = np.split(data.sample(frac=1, random_state=42), [int(.7*len(data)), int(.9*len(data))])

In [167]:
type(train.iloc[0,2])

numpy.float64

In [6]:
Vocab_char = {}
Vocab_char[0] = '<unk>'
i=1
for char in train['Y'].unique():
    Vocab_char[i] = char
    i+=1

def get_key(val):
    for key, value in Vocab_char.items():
         if val == value:
             return key

def preprocess_data(df):
    rv = []
    records = df.loc[:, 'care_p':'sanctity_p'].to_records(index=False)
    results = list(records)
    for i in range(len(df)):
        rv.append((df['Y'].iloc[i], results[i]))
    return rv

def collate_fn(batch):
    
    speech_mf = []
    labels = []
    
    for b in batch:
        label = get_key(b[0])
        labels.append(label)
        s = [i for i in b[1]]
        speech_mf.append(s)
    speech_mf = torch.tensor(speech_mf)
    labels = torch.tensor(labels)
    
    return labels, speech_mf
    

In [7]:
t1,t2 = collate_fn(test_data)

NameError: name 'test_data' is not defined

In [171]:
t2

tensor([[0.1113, 0.0873, 0.1189, 0.1097, 0.0749],
        [0.0985, 0.1028, 0.0845, 0.1074, 0.0820],
        [0.1110, 0.1058, 0.0891, 0.0836, 0.0791],
        ...,
        [0.1099, 0.1036, 0.1009, 0.0907, 0.0842],
        [0.0956, 0.1176, 0.0994, 0.0744, 0.0834],
        [0.0928, 0.0864, 0.1055, 0.0921, 0.0727]], dtype=torch.float64)

In [8]:
num_labels = len(Vocab_char)
vocab_size = 5

class NNeMFDTagger(nn.Module):
    def __init__(self, num_labels, vocab_size):

        super(NNeMFDTagger, self).__init__()
        self.linear = nn.Linear(vocab_size, num_labels)


    def forward(self, bow_vec):
        return F.log_softmax(self.linear(bow_vec), dim=1)

In [9]:
loss_function = torch.nn.NLLLoss()

def train_an_epoch(dataloader):
    model.train()
    log_interval = 500

    for idx, (label, speech_mf) in enumerate(dataloader):
        model.zero_grad()
        probs = model(speech_mf.float())
        loss = loss_function(probs, label)
        loss.backward()
        optimizer.step()
        if idx % log_interval == 0 and idx > 0:
            print(f'At iteration {idx} the loss is {loss:.3f}.')

In [10]:
def get_accuracy(dataloader):
    model.eval()
    with torch.no_grad():    
        total_acc, total_count = 0, 0
        for idx, (label, speech_mf) in enumerate(dataloader):
            log_probs = model(speech_mf.float())
            total_acc += (log_probs.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [11]:
BATCH_SIZE = 64 
  
train_data = preprocess_data(train)
valid_data = preprocess_data(valid)
test_data = preprocess_data(test)

train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE,
                              shuffle=True, 
                              collate_fn=collate_fn)
valid_dataloader = DataLoader(valid_data, batch_size=BATCH_SIZE,
                              shuffle=False, 
                              collate_fn=collate_fn)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE,
                             shuffle=False, 
                             collate_fn=collate_fn)

In [12]:
model = NNeMFDTagger(len(Vocab_char),5)

In [ ]:
import time
import matplotlib.pyplot as plt
%matplotlib inline

EPOCHS = 3 # epoch
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

accuracies=[]
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train_an_epoch(train_dataloader)
    accuracy = get_accuracy(valid_dataloader)
    accuracies.append(accuracy)
    time_taken = time.time() - epoch_start_time
    print(f'Epoch: {epoch}, time taken: {time_taken:.1f}s, validation accuracy: {accuracy:.3f}.')
    
plt.plot(range(1, EPOCHS+1), accuracies)

Epoch: 1, time taken: 0.3s, validation accuracy: 0.008.
Epoch: 2, time taken: 0.3s, validation accuracy: 0.029.
